## TESTING NOTEBOOK FOR REPAIRING PDF'S

In [3]:
from pathlib import Path
import deepdoctection as dd
import spacy


In [4]:
config_overwrite = ["LANGUAGE='nld'","TEXT_ORDERING.INCLUDE_RESIDUAL_TEXT_CONTAINER=True"]
analyzer = dd.get_dd_analyzer(config_overwrite=config_overwrite)

nlp = spacy.load("nl_core_news_sm")


/home/marty/miniconda3/envs/PDFair/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0215 12:28.52 @dd.py:422]  INF  Config: 
 {'DEVICE': 'cpu',
 'LANGUAGE': 'nld',
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg16_bn/tf/crnn_vgg16_bn-76b7f2c6.zip'},
                     'DOCTR_WORD': {'PT': 'doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt',
                                    'TF': 'doctr/db_resnet50/tf/db_resnet50-adcafc63.zip'}}},
 'PT': {'CELL

In [472]:
FILE = "evaluation/pdfs/tk-beslisnota-bij-kamerbrief-inzake-verdeelbesluiten"
# FILE = "evaluation/pdfs/Adviesaanvraag+inzake+de+civiel-militaire+ontwikkelingen+in+het+ruimtedomein.pdf"

path = Path.cwd() / (FILE + ".pdf")

df = analyzer.analyze(path=path)


In [473]:
doc = iter(df)
pdf = [page for page in doc]


[0215 16:07.52 @doctectionpipe.py:84]  INF  Processing tk-beslisnota-bij-kamerbrief-inzake-verdeelbesluiten_0.pdf
[0215 16:07.59 @context.py:126]  INF  ImageLayoutService total: 5.5183 sec.
[0215 16:07.59 @context.py:126]  INF  SubImageLayoutService total: 0.0001 sec.
[0215 16:07.59 @context.py:126]  INF  SubImageLayoutService total: 0.0001 sec.
[0215 16:07.59 @context.py:126]  INF  TableSegmentationService total: 0.0001 sec.
[0215 16:07.59 @context.py:126]  INF  TableSegmentationRefinementService total: 0.0 sec.
[0215 16:08.01 @context.py:126]  INF  TextExtractionService total: 2.282 sec.
[0215 16:08.01 @context.py:126]  INF  MatchingService total: 0.0049 sec.
[0215 16:08.01 @context.py:126]  INF  TextOrderService total: 0.0242 sec.
[0215 16:08.02 @doctectionpipe.py:84]  INF  Processing tk-beslisnota-bij-kamerbrief-inzake-verdeelbesluiten_1.pdf
[0215 16:08.08 @context.py:126]  INF  ImageLayoutService total: 5.5606 sec.
[0215 16:08.08 @context.py:126]  INF  SubImageLayoutService total:

In [474]:
for layout in sorted(pdf[0].layouts, key=lambda x: x.reading_order):
    print(f"{layout.reading_order:2d} - {layout.category_name:5s} : {layout.text}")


 1 - line  : OA
 2 - line  : Tweede Kamer
 3 - title : nota
 4 - text  : Beslisnota Kamerbrief inzake verdeelbesluiten
 5 - title : 1 Aanleiding
 6 - text  : Op 31 oktober 2024 ontving u de verslagen van de commissarissen van de Koning in het kader van de ‘Wet gemeentelijke taak mogelijk maken asielopvang- voorzieningen’ (hierna: de wet). Deze heeft u op 1 november jl. aan de Kamer gezonden. In de verslagen doen de commissarissen verslag van het overleg tussen gemeenten, provincie en het COA over de uitvoering van de taken op grond van de wet en geven ze aan hoe de verdeling van reguliere en amv- opvangplaatsen voor de daaropvolgende twee jaren er uit zal zien. Op grond van artikel 5, eerste lid, van de wet neemt u voor 31 december 2024 een verdeelbesluit per provincie. In deze besluiten worden de opvangplaatsen per provincie, zoals bepaald in de provinciale opvangopgave die voor 1 februari van hetzelfde kalenderjaar bekend is gemaakt, verdeeld over de in het besluit aangewezen gemeent

In [475]:
!pdftotext {FILE+".pdf"} -nopgbrk -l 1

file = open(FILE+".txt")
text = file.read()
file.close()


In [476]:
class Word:
    def __init__(self, token, text, line, index):
        self.token = token
        self.text = text

        self.line = line
        self.index = index

        self.missing = 0

    def __repr__(self):
        return f"{self.text} = {self.token.text}({self.line}:{self.index}) - missing {self.missing} times."


In [477]:
# PdfToText
ptt = {}
n = 4    # N-gram (4)

def doc_get_word_1(doc):
    for token in doc:
        if not token.is_punct:
            return token
    return None

tokens = [Word(doc_get_word_1(nlp.make_doc(t)), t, l, i) for l, s in enumerate(text.split("\n")) for i, t in enumerate(s.split(" ")) if t]
tokens = [t for t in tokens if t.token]

ptt['tokens'] = tokens
ptt['ngrams'] = {" ".join(t.token.text for t in tokens[i:i+n]) : tokens[i:i+n] for i in range(len(tokens) - n + 1)}


In [478]:
# DeepDocTection
ddt = {}

tokens = [token for token in nlp.make_doc(pdf[0].text.replace("\n", " ")) if not token.is_punct]

ddt['tokens'] = tokens
ddt['ngrams'] = {" ".join(t.text for t in tokens[i:i+n]) for i in range(len(tokens) - n + 1)}


In [479]:
missing = [ngram for ngram in list(ptt['ngrams'].keys()) if not ngram in ddt['ngrams']]

for ngram in missing:
    for token in ptt['ngrams'][ngram]:
        token.missing += 1
        print(token)


Tweede = Tweede(0:0) - missing 1 times.
Kamer = Kamer(0:1) - missing 1 times.
Directoraat-Generaal = Directoraat-Generaal(2:0) - missing 1 times.
Migratie = Migratie(3:0) - missing 1 times.
Kamer = Kamer(0:1) - missing 2 times.
Directoraat-Generaal = Directoraat-Generaal(2:0) - missing 2 times.
Migratie = Migratie(3:0) - missing 2 times.
Directie = Directie(4:0) - missing 1 times.
Den = Den(9:2) - missing 1 times.
Haag = Haag(9:3) - missing 1 times.
www.rijksoverheid.nl/jenv = www.rijksoverheid.nl/jenv(10:0) - missing 1 times.
Beslisnota = Beslisnota(12:0) - missing 1 times.
Haag = Haag(9:3) - missing 2 times.
www.rijksoverheid.nl/jenv = www.rijksoverheid.nl/jenv(10:0) - missing 2 times.
Beslisnota = Beslisnota(12:0) - missing 2 times.
Kamerbrief = Kamerbrief(12:1) - missing 1 times.
www.rijksoverheid.nl/jenv = www.rijksoverheid.nl/jenv(10:0) - missing 3 times.
Beslisnota = Beslisnota(12:0) - missing 3 times.
Kamerbrief = Kamerbrief(12:1) - missing 2 times.
inzake = inzake(12:2) - miss

In [480]:
from bs4 import BeautifulSoup


In [481]:
html = BeautifulSoup(f'<html></html>', 'html.parser')

head = html.new_tag("head")
title = html.new_tag("title")
title.string = f"Evaluation of {FILE}"
head.append(title)
html.html.append(head)
    

In [482]:
line = 0

body = html.new_tag("body")
p = html.new_tag("p")

colors = ["white","rgba(255, 255, 0, 0.5)","rgba(255, 166, 0, 0.5)","rgba(255, 0, 0, 0.5)","rgba(139, 0, 0, 0.5)"]

for token in ptt['tokens']:
    if token.line != line:
        for i in range(token.line - line):
            p.append(html.new_tag("br"))
        line = token.line

    span = html.new_tag("span", style=f"background-color: {colors[token.missing]};")
    span.string = token.text
    p.append(span)

body.append(p)
html.html.append(body)



In [483]:
with open(f"{FILE+'.html'}", "w", encoding="utf-8") as file:
    file.write(html.prettify())


- footnote / links
- newline formatting
- gemarkeerde tekst wordt verwijderd
- lists worden niet correct gehandeld. `e` i.p.v. bullet point 
- Titels worden vergeten (`Naschrift advies`)